# 14.4 Running A Thread

OpenAI의 assistant로도 함수를 정의하고 LLM에게 이 함수를 실행하라고 말해 줄 수 있다.

아래의 코드처럼 함수를 정의하고, 함수를 설명만 해주면 된다.

우리가 QuizGPT를 만들때 배웠던 방식처럼 Schema를 만들어 주기만 하면 된다. (quizGPT_9.8.ipynb 참고)

In [136]:
from langchain.utilities.google_search import GoogleSearchAPIWrapper
import yfinance
import json
import os

google_api_key = os.environ.get("GOOGLE_API_KEY")
google_cse_id = os.environ.get("GOOGLE_CSE_ID")

def get_ticker(inputs):
    google_search = GoogleSearchAPIWrapper(
        google_cse_id=google_cse_id,
        google_api_key=google_api_key,
    )
    company_name = inputs["company_name"]
    return google_search.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())
    # 여기서 dumps의 역할은 object를 받아서 json포맷의 string으로 바꿔주는 역할을 한다.

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())
    # 여기서 period="3mo"라는 말은 마지막 3개월 이라는 뜻이다.

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

# get_income_statement({"ticker": "AAPL"})

이렇게 만들고 나면, 우리는 assistant를 만들어 주어야 한다.

아래처럼 해주면 된다.

In [137]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help them decide if they should buy the stock or not.",
#     model="gpt-4o",
#     tools=functions,
# )
# 어시스턴트를 하나 더 만들지 않게 하기 위해 주석처리함.


이렇게 되면 assistant가 만들어 졌고, 이는 "https://platform.openai.com/assistants/"에서 확인이 가능하다.

여기서 또한 Assistant Id도 확인이 가능하다.

In [138]:
assistant_id = "asst_CJaj6ZDRFCtl2nPDls1IBTZk"

이렇게 Assistant Id는 계속 필요하니까 변수에 넣어두자.

In [139]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_4qe4akln5aUjWBKyBWQ1HK5w', created_at=1740750153, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

이제 Thread를 만들어서 메세지를 넣어주어야 한다.

Open AI의 Assistant의 작동방식을 상기해보자.

먼저 Assistant를 만들고, Thread를 만들어 메세지를 넣는다.  
그다음 run을 하여 LLM에게 보낸다.

이런식으로 작동한다는 것을 잊지말자.

In [140]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )
# 이 함수를 만드는 이유는 run의 status를 반복적으로 받아와서 상태를 확인하기 위해서이다.
# 즉 run이 어떻게 되어가고 있는지, 어떤 상태인지를 retrieve를 통해 받아오기 위해 만들어진 함수이다.

우리는 run을 하고 나서 run의 상태를 모른다. 따라서 run의 상태를 알기 위해 위와 같은 함수를 만들었다.

retrieve로 상태를 받아올 수 있다.

##### run이 가질수 있는 상태
1. queued: 우리가 run을 만드는(create) 순간에는 queued(대기)상태가 된다.
2. in_progress: 그 다음으로는 In progress(진행중) 상태가 된다. 우리가 requires action이란 상태에서 tool의 실행 결과를 submit해도 in progress(진행중)상태가 된다.
3. cancelling -> cancelled: 우리가 취소를 하면 cancelled(취소) 상태가 된다.
4. requires action: 만약 어떠한 함수를 실행하여 그 반환값을 받아와야 되는 것이면 requires action(조치가 필요함)이라는 상태가 된다. 즉, assistant가 우리에게 tool을 실행하기를 요청하는 상태이다.
5. failed: 어떤 예상치 못한 일이 발생해서 failed(실패)된 상태.
6. completed: tool으로부터 반환값을 잘 받아와서 assistant가 completed(완료)된 상태.
7. expired: tool의 반환값(output)을 submit하는데 너무 오래 걸렸다면 expired(만료)된 상태가 된다.

In [141]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_TEmzt9dbDhyj9Y7KVt27mjyI', assistant_id='asst_CJaj6ZDRFCtl2nPDls1IBTZk', cancelled_at=None, completed_at=None, created_at=1740750175, expires_at=1740750775, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help them decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_4qe4akln5aUjWBKyBWQ1HK5w', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(n

이렇게 create로 run을 만들 수(created) 있다.

이제 run을 만들었으니 이 run의 상태를 받아와서 살펴보자.

In [142]:
get_run(run.id, thread.id)

Run(id='run_TEmzt9dbDhyj9Y7KVt27mjyI', assistant_id='asst_CJaj6ZDRFCtl2nPDls1IBTZk', cancelled_at=None, completed_at=None, created_at=1740750175, expires_at=1740750775, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help them decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_qaC7ykwhQmmemP0ve4MMxVOB', function=Function(arguments='{"company_name":"Salesforce"}', name='get_ticker'), type='function')]), type='submit_tool_outputs'), response_format='auto', started_at=1740750177, status='requires_action', thread_id='thread_4qe4akln5aUjWBKyBWQ1HK5w', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='G

상태만 나오는 것이 아니라 run의 정보가 모두 나온다. 여기중에서 status(상태)만 골라서 보면 된다.

In [143]:
get_run(run.id, thread.id).status

'requires_action'

In [144]:
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id
    )
    for message in messages:
        print(message)

LLM이 보내온 메세지가 무엇인지, 우리가 보낸 메세지는 무엇인지를 알아보기 위해 함수를 만들었다.

In [145]:
get_messages(thread.id)

Message(id='msg_7uqKZUSWGrWYGG7rDGzmMFQp', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know if the Salesforce stock is a good buy'), type='text')], created_at=1740750153, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_4qe4akln5aUjWBKyBWQ1HK5w')


```pgsql
Message(
    id='msg_Pk3UWkVy0s2jAk398KKNGo7g',
    assistant_id=None,
    attachments=[],
    completed_at=None,
    content=[
        TextContentBlock(
            text=Text(
                annotations=[],
                value='I want to know if the Salesforce stock is a good buy'
            ),
            type='text'
        )
    ],
    created_at=1740749411,
    incomplete_at=None,
    incomplete_details=None,
    metadata={},
    object='thread.message',
    role='user',
    run_id=None,
    status=None,
    thread_id='thread_icVZFPu7fqir8DcCyZec1zdB'
)
```


In [146]:
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

user가 먼저 올 수 있도록 함수를 조금 수정했다.  
또한 딱 메세지만 보일 수 있도록 또 수정했다.